In [3]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence


import os

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['HF_API_KEY'] = os.getenv('HF_API_KEY')


In [16]:
prompt1 = PromptTemplate(
    template="Write a joke about {topic}",
    input_variables=["topic"],
)

In [17]:
prompt2 = PromptTemplate(
    template="explain the following joke {joke}",
    input_variables=["joke"],
)

In [18]:
model = ChatOpenAI(
    temperature=0.7,
    model="gpt-3.5-turbo",
    openai_api_key=os.getenv('OPENAI_API_KEY')
)

In [19]:
parser = StrOutputParser()


In [21]:
chain = RunnableSequence(prompt1, model, parser, prompt2,model, parser)

In [22]:
chain.invoke({"topic": "RCB in IPL"})

"This joke is a play on words, as it uses the idea of a magician making things disappear to humorously suggest that the Royal Challengers Bangalore (RCB) cricket team needs someone to make their hopes of winning disappear. This is a joke about RCB's history of not performing well in the Indian Premier League (IPL) and their struggles to win the tournament. The joke implies that their hopes of winning are so low that they need a magician to make them disappear completely."

In [23]:
from langchain.schema.runnable import RunnableParallel

In [ ]:
prompt1 = PromptTemplate(
    template="Generate a tweet about topic{topic}",
    input_variables=["topic"],
)

In [26]:
prompt2 = PromptTemplate(
    template="Generate a linkedin post bout topic {topic}",
    input_variables=["topic"],
)

In [28]:
parallell_chain = RunnableParallel(
    {'tweet': RunnableSequence(prompt1, model, parser),
    'linkedin': RunnableSequence(prompt2, model, parser)}
    )

In [29]:
parallell_chain.invoke({"topic": "RCB in IPL"})

{'tweet': '"RCB is on fire this IPL season! 🔥 With Kohli leading the charge and AB de Villiers smashing boundaries, they are definitely a team to watch out for. #PlayBold #RCB #IPL2021"',
 'linkedin': 'Excited to see Royal Challengers Bangalore in action in the upcoming IPL season! With a strong squad and a talented captain like Virat Kohli leading the way, I have no doubt that RCB will put up a great show on the field. Wishing them all the best for a successful campaign. #RCB #IPL2022 #CricketFever 🏏🔥'}

In [31]:
from langchain.schema.runnable import RunnablePassthrough

In [32]:
prompt1 = PromptTemplate(
    template="Write a joke about {topic}",
    input_variables=["topic"],
)

prompt2 = PromptTemplate(
    template="explain the following joke {joke}",
    input_variables=["joke"],
)

In [36]:
joke_generation_chain = RunnableSequence(prompt1, model, parser)


In [38]:
parallell_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'explain': RunnableSequence(prompt2, model, parser)
    })

In [39]:
final_chain = RunnableSequence(
    joke_generation_chain,
    parallell_chain
)

In [40]:
final_chain.invoke({"topic": "RCB in IPL"})

{'joke': 'Why did the RCB player bring a ladder to the IPL match?\n\nBecause they heard they needed to climb to reach the top of the table!',
 'explain': "This joke is referencing the fact that the Royal Challengers Bangalore (RCB) cricket team has historically struggled in the Indian Premier League (IPL) and has not been able to secure a top position in the tournament standings. By bringing a ladder to the match, it implies that the RCB player believes they need to physically climb up to the top of the table in order to improve their position in the league. It is meant to be a humorous way of poking fun at the team's lack of success in the tournament."}

In [41]:
from langchain.schema.runnable import RunnableLambda

In [42]:
def word_count(text: str) -> int:
    return len(text.split())

In [43]:
runnable_word_counter = RunnableLambda(func=word_count)

In [44]:
runnable_word_counter.invoke("This is a test string to count the number of words in it.")

13

In [46]:
parallell_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'word_count': RunnableLambda(func=word_count)})

In [48]:
final_chain = RunnableSequence(
    joke_generation_chain,
    parallell_chain
)

In [49]:
final_chain.invoke({"topic": "RCB in IPL"})

{'joke': 'Why did the RCB player bring a ladder to the IPL match?\n\nBecause they heard they needed to raise their game!',
 'word_count': 21}

In [50]:
from langchain.schema.runnable import RunnableBranch

In [58]:
prompt1 = PromptTemplate(
    template="Write a detailed report on {topic}",
    input_variables=["topic"],
)

prompt2 = PromptTemplate(
    template="Summarize the following text {text}",
    input_variables=["text"],
)

In [67]:
report_gen_chain = prompt1 | model | parser

In [68]:
branch_chain = RunnableBranch(
    (lambda x: len(x.split())>100, RunnableSequence(prompt2, model, parser) ),
    RunnablePassthrough()
)

In [69]:
final_chain = RunnableSequence(
    report_gen_chain,
    branch_chain
)

In [70]:
final_chain.invoke({"topic": "why RCB in not winning  IPL"})

"The text discusses the struggles faced by the Royal Challengers Bangalore (RCB) in the Indian Premier League (IPL). Despite having star players like Virat Kohli and AB de Villiers, RCB has failed to win the tournament due to inconsistent team performances, over-reliance on key players, weaknesses in their bowling attack, and questionable management decisions. Unless these issues are addressed, RCB's chances of winning the IPL in the near future remain slim."